<a href="https://colab.research.google.com/github/kategavrishina/info-search/blob/master/HW5/hw5_ml.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Семинар и дз
Сделать и улучшить любую ML-модель на ваших проектных данных (просто клф, бленд, ранжирование, что-то что вы придумали сами...), используя любые признаки, какие захотите. Оцениваться будут:
* факт выполнения задания :)
* корректность кода (чтобы код не падал) и отсутствие логических ошибок (e.g. затестили на трейне)
* итеративность улучшения (например взяли один сет признаков, показали качество; потом добавили / подкрутили / использовали другую модель, показали качество...)
* креативность признаков
* аккуратность ноутбука

Дедлайн: 15 октября

In [1]:
! pip install pymorphy2

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 8.2MB 5.9MB/s 


In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
from nltk.corpus import stopwords

In [4]:
russian_stopwords = stopwords.words('russian')

In [5]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [6]:
import re

In [7]:
def preprocess(text):
    reg = re.compile('[^а-яА-Я ]')
    text = reg.sub('', text.lower().strip())
    words = text.strip().split()
    s = []
    for word in words:
        if word != '':
            word = morph.parse(word)[0].normal_form
            if word not in russian_stopwords:
                s.append(word)
    return s

In [8]:
import numpy as np

In [9]:
import pandas as pd

In [10]:
ans = pd.read_excel('answers_base.xlsx')

In [11]:
answers = pd.DataFrame(ans[['Номер связки', 'Текст ответа']])

In [12]:
ans['Вопросы'] = ans['Текст вопросов'].apply(lambda text: text.split('\n'))

In [13]:
ans = ans.explode('Вопросы')

In [14]:
ans.reset_index(drop=True, inplace=True)
ans.drop(['Текст вопросов', 'Текст ответа', 'Тематика'], axis=1, inplace=True)

In [15]:
ans.head()

,Номер связки,Вопросы
0,57,У ребенка в школе продлили каникулы. Могу ли я...
1,57,Больничный лист?
2,57,"Есть ли компенсация, в случае если есть разниц..."
3,57,как оплачивается больничный при коронавирусе?
4,57,"Я контактный, дадут ли больничный?"


In [16]:
q = pd.read_excel('queries_base.xlsx', usecols='A, B', names=['Вопросы', 'Номер связки'])

In [17]:
q = q[['Номер связки', 'Вопросы']]

In [18]:
data = pd.concat([ans, q], ignore_index=True)

In [19]:
from tqdm.auto import tqdm

tqdm.pandas()

In [20]:
data['Preprocessed'] = data['Вопросы'].progress_apply(lambda text: ' '.join(preprocess(str(text))))

In [21]:
data.drop(data[data['Номер связки'].isna()].index, inplace=True)

In [22]:
data.drop(data[data['Preprocessed'] == ''].index, inplace=True)

In [23]:
data.reset_index(inplace=True, drop=True)

In [24]:
data_train, data_test = data[:int(len(data)*0.7)], data[int(len(data)*0.7):]

### Сначала попробуем с преобразованием в Tfidf-матрицу

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
tfidf = TfidfVectorizer(min_df=5)

X_train = tfidf.fit_transform(data_train.Preprocessed)
X_test = tfidf.transform(data_test.Preprocessed)

#### Случайный лес

In [29]:
rf1 = RandomForestClassifier()
rf1.fit(X_train, data_train['Номер связки'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [30]:
pred_rf1 = rf1.predict(X_test)

In [31]:
acc_rf1 = accuracy_score(data_test['Номер связки'], pred_rf1)
acc_rf1

0.6568627450980392

In [32]:
rf2 = RandomForestClassifier(max_depth=5,
                             n_estimators=150,
                             min_samples_split=3)
rf2.fit(X_train, data_train['Номер связки'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=3,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [33]:
pred_rf2 = rf2.predict(X_test)

In [34]:
acc_rf2 = accuracy_score(data_test['Номер связки'], pred_rf2)
acc_rf2

0.4738562091503268

Попробуем использовать подбор параметров при помощи кросс-валидации

In [35]:
from sklearn.model_selection import GridSearchCV

In [36]:
parameters = {'max_depth':[None, 10, 15],
              'n_estimators':[100, 150, 200],
              'min_samples_split':[2, 3, 4]}
clf = GridSearchCV(rf1, parameters)
clf.fit(X_train, data_train['Номер связки'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [37]:
clf.best_params_

{'max_depth': None, 'min_samples_split': 4, 'n_estimators': 150}

In [38]:
pred_cv = clf.predict(X_test)

In [39]:
acc_cv = accuracy_score(data_test['Номер связки'], pred_cv)
acc_cv

0.6503267973856209

#### Градиентный бустинг

In [40]:
from sklearn.ensemble import GradientBoostingClassifier

In [41]:
gb1 = GradientBoostingClassifier()
gb1.fit(X_train, data_train['Номер связки'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [42]:
pred_gb1 = gb1.predict(X_test)

In [43]:
acc_gb1 = accuracy_score(data_test['Номер связки'], pred_gb1)
acc_gb1

0.6154684095860566

In [ ]:
parameters = {'max_depth':[3, 5, 10],
              'learning_rate':[0.1, 0.01, 0.001],
              'min_samples_split':[2, 3, 4]}
clf2 = GridSearchCV(gb1, parameters)
clf2.fit(X_train, data_train['Номер связки'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=GradientBoostingClassifier(ccp_alpha=0.0,
                                                  criterion='friedman_mse',
                                                  init=None, learning_rate=0.1,
                                                  loss='deviance', max_depth=3,
                                                  max_features=None,
                                                  max_leaf_nodes=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                                  n_estimators=100,
                                                  n_iter_no_change=None,
      

In [ ]:
clf2.best_params_

{'max_depth': None, 'min_samples_split': 3, 'n_estimators': 200}

In [ ]:
pred_cv2 = clf2.predict(X_test)

In [ ]:
acc_cv2 = accuracy_score(data_test['Номер связки'], pred_cv2)
acc_cv2

0.6503267973856209

#### Логистическая регрессия

In [44]:
from sklearn.linear_model import LogisticRegression

In [45]:
lr1 = LogisticRegression()
lr1.fit(X_train, data_train['Номер связки'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
pred_lr1 = lr1.predict(X_test)

In [47]:
acc_lr1 = accuracy_score(data_test['Номер связки'], pred_lr1)
acc_lr1

0.6753812636165577

In [65]:
parameters = {
    'solver':['newton-cg', 'lbfgs', 'sag', 'saga'],
    'C':[1.0, 1.5, 2.0]
}
lr2 = GridSearchCV(lr1, parameters)

In [66]:
lr2.fit(X_train, data_train['Номер связки'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': [1.0, 1.5, 2.0],
                         'solver': ['newton-cg', 'lbfgs', 'sag', 'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [67]:
pred_lr2 = lr2.predict(X_test)

In [68]:
lr2.best_params_

{'C': 2.0, 'solver': 'saga'}

In [69]:
acc_lr2 = accuracy_score(data_test['Номер связки'], pred_lr2)
acc_lr2

0.6808278867102396

#### Результаты

In [141]:
tfidf_res = pd.DataFrame({'method':['RF', 'RF + params', 'RF + grid_search', 'GB', 'GB + grid_search', 'LR', 'LR + grid_search'],
                          'accuracy':[acc_rf1, acc_rf2, acc_cv, acc_gb1, acc_cv2, acc_lr1, acc_lr2]})

In [142]:
tfidf_res

,method,accuracy
0,RF,0.656863
1,RF + params,0.473856
2,RF + grid_search,0.650327
3,GB,0.615468
4,GB + grid_search,0.650327
5,LR,0.675381
6,LR + grid_search,0.680828


### Также можно попробовать векторизовать обычным Count векторайзером

In [70]:
from sklearn.feature_extraction.text import CountVectorizer

In [74]:
count = CountVectorizer()

X_train = count.fit_transform(data_train.Preprocessed)
X_test = count.transform(data_test.Preprocessed)

#### Случайный лес

In [76]:
rf1_c = RandomForestClassifier()
rf1_c.fit(X_train, data_train['Номер связки'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [77]:
pred_c_rf1 = rf1_c.predict(X_test)

In [78]:
acc_c_rf1 = accuracy_score(data_test['Номер связки'], pred_c_rf1)
acc_c_rf1

0.6372549019607843

#### Градиентный бустинг

In [92]:
gb1_c = GradientBoostingClassifier()
gb1_c.fit(X_train, data_train['Номер связки'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [94]:
pred_c_gb1 = gb1_c.predict(X_test)

In [95]:
acc_c_gb1 = accuracy_score(data_test['Номер связки'], pred_c_gb1)
acc_c_gb1

0.6067538126361656

#### Логистическая регрессия

In [79]:
lr1_c = LogisticRegression()
lr1_c.fit(X_train, data_train['Номер связки'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [80]:
pred_c_lr1 = lr1_c.predict(X_test)

In [81]:
acc_c_lr1 = accuracy_score(data_test['Номер связки'], pred_c_lr1)
acc_c_lr1

0.6775599128540305

In [127]:
lr2_c = LogisticRegression(C=1.5, solver='sag')
lr2_c.fit(X_train, data_train['Номер связки'])

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [128]:
pred_c_lr2 = lr2_c.predict(X_test)

In [129]:
acc_c_lr2 = accuracy_score(data_test['Номер связки'], pred_c_lr2)
acc_c_lr2

0.6742919389978214

#### Результаты

In [143]:
count_res = pd.DataFrame({'method':['RF', 'GB', 'LR1',  'LR2'],
                          'accuracy':[acc_c_rf1, acc_c_gb1, acc_c_lr1, acc_c_lr2]})

In [144]:
count_res

,method,accuracy
0,RF,0.637255
1,GB,0.606754
2,LR1,0.677560
3,LR2,0.674292


### Семантические вектора при помощи предобученных эмбеддингов

In [133]:
%load_ext autoreload

from gensim.models import KeyedVectors

In [134]:
import urllib

In [135]:
urllib.request.urlretrieve("https://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextcbow_300_5_2018.tgz", "araneum_none_fasttextcbow_300_5_2018.tgz")

('araneum_none_fasttextcbow_300_5_2018.tgz',
 <http.client.HTTPMessage at 0x7f56aff16320>)

In [136]:
! tar xvfz araneum_none_fasttextcbow_300_5_2018.tgz

araneum_none_fasttextcbow_300_5_2018.model
araneum_none_fasttextcbow_300_5_2018.model.vectors_ngrams.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors.npy
araneum_none_fasttextcbow_300_5_2018.model.vectors_vocab.npy


In [137]:
model_file = 'araneum_none_fasttextcbow_300_5_2018.model'
model = KeyedVectors.load(model_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [138]:
def normalize_vec(v):
     return v / np.sqrt(np.sum(v ** 2))

In [131]:
def get_vector_mean(lemmas):
    lemmas_vectors = np.zeros((len(lemmas), model.vector_size))
    vec = np.zeros((model.vector_size,))

    for idx, lemma in enumerate(lemmas):
        try:
            lemmas_vectors[idx] = model[lemma]
        except AttributeError:
            continue

    if lemmas_vectors.shape[0] is not 0:
        vec = np.mean(lemmas_vectors, axis=0)

    if np.sum(vec ** 2) != 0:
        vec = normalize_vec(vec)
    
    return vec

In [139]:
matrix = np.zeros((len(data), model.vector_size))
for idx, text in enumerate(data.Preprocessed):
    matrix[idx] = get_vector_mean(text.split())

In [140]:
matrix_train, matrix_test = matrix[:int(len(data)*0.7)], matrix[int(len(data)*0.7):]

#### Случайный лес

In [145]:
rf1_m = RandomForestClassifier()
rf1_m.fit(matrix_train, data_train['Номер связки'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [146]:
pred_m_rf1 = rf1_m.predict(matrix_test)

In [147]:
acc_m_rf1 = accuracy_score(data_test['Номер связки'], pred_m_rf1)
acc_m_rf1

0.5860566448801743

In [148]:
parameters = {'max_depth':[None, 10, 15],
              'n_estimators':[100, 150, 200],
              'min_samples_split':[2, 3, 4]}
rf2_m = GridSearchCV(rf1_m, parameters)
rf2_m.fit(matrix_train, data_train['Номер связки'])

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [149]:
rf2_m.best_params_

{'max_depth': None, 'min_samples_split': 3, 'n_estimators': 200}

In [150]:
pred_m_rf2 = rf2_m.predict(matrix_test)

In [151]:
acc_m_rf2 = accuracy_score(data_test['Номер связки'], pred_m_rf2)
acc_m_rf2

0.6034858387799564

#### Градиентный бустинг

In [152]:
gb1_m = GradientBoostingClassifier()
gb1_m.fit(matrix_train, data_train['Номер связки'])

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [153]:
pred_m_gb1 = gb1_m.predict(matrix_test)

In [154]:
acc_m_gb1 = accuracy_score(data_test['Номер связки'], pred_m_gb1)
acc_m_gb1

0.5697167755991286

#### Логистическая регрессия

In [155]:
lr1_m = LogisticRegression()
lr1_m.fit(matrix_train, data_train['Номер связки'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [156]:
pred_m_lr1 = lr1_m.predict(matrix_test)

In [157]:
acc_m_lr1 = accuracy_score(data_test['Номер связки'], pred_m_lr1)
acc_m_lr1

0.6677559912854031

In [164]:
lr2_m = LogisticRegression(C=3.5, solver='sag')
lr2_m.fit(matrix_train, data_train['Номер связки'])

LogisticRegression(C=3.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)

In [165]:
pred_m_lr2 = lr2_m.predict(matrix_test)

In [166]:
acc_m_lr2 = accuracy_score(data_test['Номер связки'], pred_m_lr2)
acc_m_lr2

0.6797385620915033

#### Результаты

In [191]:
matrix_res = pd.DataFrame({'method':['RF', 'RF + grid_search', 'GB', 'LR1',  'LR2'],
                          'accuracy':[acc_m_rf1, acc_m_rf2, acc_m_gb1, acc_m_lr1, acc_m_lr2]})

In [192]:
matrix_res

,method,accuracy
0,RF,0.586057
1,RF + grid_search,0.603486
2,GB,0.569717
3,LR1,0.667756
4,LR2,0.679739
